## Groups Assign to Capacity

This API will assign the given group to the specificed capacity id.

### Notes:

1. This code assumes that there is an existing capacity created that the service principal is an administrator of (CreateCapacity.ipynb) and there is a Power BI Workspace (aka Group) created that the service principal is an administator of (workspace/CreateWorkspace.ipynb)
1. The API Documentation can be found here: https://learn.microsoft.com/en-us/rest/api/power-bi/capacities/groups-assign-to-capacity

In [ ]:
pip install requests msal

In [ ]:
#For orgaization purposes I put notbooks in subfolders not the root of the proejct.aad_token
#This code adds the root directory of the project to the sys path so we can load class modules from the services folder
#I think this only needs to be run once, but including it for completeness.
import os, sys
projectRoot = os.path.abspath('.')
directory = os.path.dirname(projectRoot)
if not directory in sys.path: sys.path.append(directory)

In [ ]:
#This leverages the code encapsulated in services/aadservice.py that encapsulates the service principle login
from services.aadservice import AadService
aadToken = AadService.get_access_token()
#aadCredential ("https://graph.microsoft.com/.default")

headers =  {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + aadToken}

In [ ]:
import requests
import json
from services.env import const

apiUrl = f'https://api.powerbi.com/v1.0/myorg/capacities'      
capacityId = ""

apiResponse = requests.get(apiUrl, headers=headers)

if apiResponse.status_code != 200:
    description = f'Error creating workspace:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
else:
    apiResponse = json.loads(apiResponse.text)
    capacities = apiResponse["value"]
    for capacity in capacities:
        if capacity["displayName"] == const.capacityName:
            capacityId = capacity["id"]

print(f"The capacity {const.capacityName} has id {capacityId}")

In [ ]:
import requests
import json

apiUrl = f'https://api.powerbi.com/v1.0/myorg/groups/'       
workspaceId = ""

apiResponse = requests.get(apiUrl, headers=headers)
#error handling for createTemporaryUplodadLocation
if apiResponse.status_code != 200:
    description = f'Error creating workspace:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
else:
    apiResponse = json.loads(apiResponse.text)
    workspaces = apiResponse["value"]
    for workspace in workspaces:
        if workspace["name"] == const.workspaceName:
            workspaceId = workspace["id"]
            
print(f"WorkspaceId is {workspaceId} ")

In [ ]:
import requests
import json

api_url = f'https://api.powerbi.com/v1.0/myorg/groups/{workspaceId}/AssignToCapacity'      

body = {'capacityId': capacityId}

api_response = requests.post(api_url, headers=headers, data=json.dumps(body))

#error handling for capacity assignment.
if api_response.status_code != 200:
    description = f'Error assigning capacity:\n  -Status Code:\t{api_response.status_code}\n  -Reason:\t{api_response.reason}\n  -RequestId:\t{api_response.headers.get("RequestId")}\n  -Text:\t{api_response.text}'
    print(description)
else:
    print("Capacity Assigned")